# Projekt: Stundenplan
June 2023, Timo Koehler

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


### Solving a School Scheduling Problem
This combinatorial optimization problem is solved using the SAT-Solver from Googles OR-Tools. The implementation is based on the ```school_scheduling_sat.py``` example from ortools: <a href="https://github.com/google/or-tools/blob/main/examples/contrib/school_scheduling_sat.py">View source on GitHub</a>. All the constraints are of course school specific and have been modeled in close cooperation with the school. 

In [410]:
%pip install ortools

Note: you may need to restart the kernel to use updated packages.


In [411]:
%pip install pandas
%pip install tabulate

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# 1. Dataloader Class



In [412]:
from ortools.sat.python import cp_model


SCHOOL_HOUR = 45  # [min]

class SchoolSchedulingProblem:
    """Dataloader class reading the input data."""

    def __init__(
        self,
        levels,
        subjects,
        curriculum,
        teachers,
        preferred_teachers,
        preferred_teachers_time_slots,
        specialties,
        time_slots,
    ):
        self._levels = levels
        self._subjects = subjects
        self._curriculum = curriculum

        for lvl, sub in self._curriculum.keys():
            assert lvl in self._levels, f"{lvl} not in LEVELS"
            assert sub in self._subjects, f"{sub} not in SUBJECTS"

        self._teachers = teachers
        self._preferred_teachers = preferred_teachers
        self._preferred_teachers_ts = preferred_teachers_time_slots
        self._specialties = specialties

        for s in self._subjects:
            assert s in self._specialties, f"{s} is not in SPECIALITIES"
        for s, ts in self._specialties.items():
            assert s in self._subjects, f"{s} is not in SUBJECTS"
            for t in ts:
                assert t in self._teachers, f"{t} is not in TEACHERS"

        self._time_slots = time_slots

    @property
    def levels(self):
        return self._levels

    def level_idx(self, level):
        return self._levels.index(level)

    @property
    def subjects(self):
        return self._subjects

    def subject_idx(self, subject):
        return self._subjects.index(subject)

    @property
    def curriculum(self):
        return self._curriculum

    def curriculum_contains(self, level, subject):
        if (level, subject) in self._curriculum:
            return True
        else:
            return False

    def curriculum_time_request(self, level, subject):
        assert (
            level,
            subject,
        ) in self._curriculum, f"level/subject: '{level}'/'{subject}' not in CURRICULUM"
        return self._curriculum[level, subject]

    @property
    def teachers(self):
        return self._teachers

    def teacher_idx(self, teacher):
        return list(self._teachers.keys()).index(teacher)

    def teacher_name(self, teacher_idx):
        assert 0 <= teacher_idx < len(self._teachers)
        return list(self._teachers.keys())[teacher_idx]

    def teacher_max_time(self, teacher_idx):
        assert 0 <= teacher_idx < len(self._teachers)
        return list(self._teachers.values())[teacher_idx]

    def preferred_teacher(self, level, subject):
        assert (
            level,
            subject,
        ) in self._preferred_teachers, (
            f"level/subject: '{level}'/'{subject}' not in PREFERRED_TEACHERS"
        )
        return self._preferred_teachers.get((level, subject))

    def preferred_teacher_time_slots(self, level, subject, time_slot):
        assert (
            level,
            subject,
            time_slot,
        ) in self._preferred_teachers_ts, f"level/subject/time_slot: '{level}'/'{subject}'/'{time_slot}' not in PREFERRED_TEACHERS_TIME_SLOTS"
        return self._preferred_teachers_ts.get((level, subject, time_slot))

    def time_slot_idx(self, time_slot):
        return list(self._time_slots.keys()).index(time_slot)

    @property
    def specialties(self):
        return self._specialties

    def specialtie_teachers(self, subject):
        assert subject in self._subjects, f"{subject} not in SUBJECTS"
        return self._specialties[subject]

    @property
    def time_slots(self):
        return self._time_slots

    def slot_duration(self, slot_idx):
        assert 0 <= slot_idx < len(self._time_slots)
        return list(self._time_slots.values())[slot_idx]

# 2. Solution printer class

In [413]:
class SchoolSchedulingSatSolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__solution_count = 0

    def OnSolutionCallback(self):
        print(f"Solution #{self.__solution_count}, objective: {self.ObjectiveValue()}")
        self.__solution_count += 1

# 3. CP-SAT solver class

In [414]:
import pandas as pd
from tabulate import tabulate


class SchoolSchedulingSatSolver:
    """Solver instance."""

    def __init__(self, problem=SchoolSchedulingProblem):
        # --------------------------------------------------------------------
        # Input data
        # --------------------------------------------------------------------
        self._problem = problem

        # Utilities
        num_levels = len(self._problem.levels)
        self._all_levels = range(num_levels)
        num_subjects = len(self._problem.subjects)
        self._all_subjects = range(num_subjects)
        num_teachers = len(self._problem.teachers)
        self._all_teachers = range(num_teachers)
        num_slots = len(self._problem.time_slots)
        self._all_slots = range(num_slots)

        # --------------------------------------------------------------------
        # Create the Model (CP-SAT)
        # --------------------------------------------------------------------
        self._model = cp_model.CpModel()

        # --------------------------------------------------------------------
        # Create and assign the Decision Variables
        # --------------------------------------------------------------------
        self._assignment = {}

        for lvl_idx, level in enumerate(self._problem.levels):
            for sub_idx, subject in enumerate(self._problem.subjects):
                for tch_idx, teacher in enumerate(self._problem.teachers):
                    for slt_idx, slot in enumerate(self._problem.time_slots):
                        key = (lvl_idx, sub_idx, tch_idx, slt_idx)
                        name = f"{level} S:{subject} T:{teacher} Slot:{slot}"
                        if teacher in self._problem.specialtie_teachers(subject):
                            if self._problem.curriculum_contains(level, subject):
                                self._assignment[key] = self._model.NewBoolVar(name)
                            else:
                                name = "NO DISP " + name
                                self._assignment[key] = self._model.NewIntVar(
                                    0, 0, name
                                )
                        else:
                            name = "NO DISP " + name
                            self._assignment[key] = self._model.NewIntVar(0, 0, name)
        print(f"There are {len(self._assignment)} variable assignments.")

        # --------------------------------------------------------------------
        # Constraints
        # --------------------------------------------------------------------

        # C1: CURRICULUM
        # Each level must have the quantity of classes per subject specified in the curriculum.
        for lvl_idx, level in enumerate(self._problem.levels):
            for sub_idx, subject in enumerate(self._problem.subjects):
                if self._problem.curriculum_contains(level, subject):
                    required_slots = self._problem.curriculum_time_request(level, subject)
                    self._model.Add(
                        sum(
                            self._assignment[lvl_idx, sub_idx, tch_idx, slt_idx]
                            for tch_idx in self._all_teachers
                            for slt_idx in self._all_slots
                        )
                        == required_slots
                    )

        # C2: Each Level can do at most one class at a time.
        for lvl_idx in self._all_levels:
            for slt_idx in self._all_slots:
                self._model.Add(
                    sum(
                        [
                            self._assignment[lvl_idx, sub_idx, tch_idx, slt_idx]
                            for sub_idx in self._all_subjects
                            for tch_idx in self._all_teachers
                        ]
                    )
                    <= 1
                )

        # C3: Teacher can do at most one class at a time.
        for tch_idx in self._all_teachers:
            for slt_idx in self._all_slots:
                self._model.Add(
                    sum(
                        [
                            self._assignment[lvl_idx, sub_idx, tch_idx, slt_idx]
                            for lvl_idx in self._all_levels
                            for sub_idx in self._all_subjects
                        ]
                    )
                    <= 1
                )

        # C4: Maximum work hours for each teacher.
        for tch_idx in self._all_teachers:
            self._model.Add(
                sum(
                    [
                        self._assignment[lvl_idx, sub_idx, tch_idx, slt_idx]
                        for lvl_idx in self._all_levels
                        for sub_idx in self._all_subjects
                        for slt_idx in self._all_slots
                    ]
                )
                <= self._problem.teacher_max_time(tch_idx)
            )

        # C5: PREFERRED_TEACHERS
        # Assign your preferred teacher and let the solver choose the time slot
        # with sum() == number of computed slots.
        for (level, subject), teachers in self._problem._preferred_teachers.items():
            lvl_idx = self._problem.level_idx(level)
            sub_idx = self._problem.subject_idx(subject)
            allowed_values = [self._problem.teacher_idx(name) for name in teachers]
            required_slots = self._problem.curriculum_time_request(level, subject)
            self._model.Add(
                sum(
                    [
                        self._assignment[lvl_idx, sub_idx, tch_idx, slt_idx]
                        for tch_idx in allowed_values
                        for slt_idx in self._all_slots
                    ]
                )
                == required_slots
            )

        # C6: PREFERRED_TEACHERS_TIME_SLOTS
        # Assign your preferred teacher for a fixed assignment of [level, subject, teacher, timeslot].
        for (
            level,
            subject,
            time_slot,
        ), teachers in self._problem._preferred_teachers_ts.items():
            lvl_idx = self._problem.level_idx(level)
            sub_idx = self._problem.subject_idx(subject)
            slt_idx = self._problem.time_slot_idx(time_slot)
            allowed_values = [self._problem.teacher_idx(name) for name in teachers]
            self._model.Add(
                self._assignment[lvl_idx, sub_idx, allowed_values[0], slt_idx] == 1
            )


    def print_teacher_schedule(self, tch_idx):
        table = {}
        teacher_name = self._problem.teacher_name(tch_idx)
        print(f"\nTeacher: {teacher_name}")
        total_working_hours = 0
        for slt_idx, slot in enumerate(self._problem.time_slots):
            for lvl_idx, level in enumerate(self._problem.levels):
                for sub_idx, subject in enumerate(self._problem.subjects):
                    key = (lvl_idx, sub_idx, tch_idx, slt_idx)
                    if self._solver.BooleanValue(self._assignment[key]):
                        total_working_hours += int(self._problem.slot_duration(slt_idx) / SCHOOL_HOUR)
                        table |= {slot: [level, subject]}
        print(f"Total teaching hours: {total_working_hours}")
        print(f"Unallocated teaching hours: {self._problem.teacher_max_time(tch_idx) - total_working_hours}")
        df = pd.DataFrame.from_dict(table, orient="index", columns=["class", "subject"])
        print(tabulate(df, headers="keys", tablefmt="psql"))

    def print_class_schedule(self, lvl_idx):
        table = {}
        level = self._problem.levels[lvl_idx]
        print(f"\nClass: {level}")
        total_working_hours = {}
        for sub in self._problem.subjects:
            total_working_hours[sub] = 0
        for slt_idx, slot in enumerate(self._problem.time_slots):
            for tch_idx, teacher in enumerate(self._problem.teachers):
                for sub_idx, subject in enumerate(self._problem.subjects):
                    key = (lvl_idx, sub_idx, tch_idx, slt_idx)
                    if self._solver.BooleanValue(self._assignment[key]):
                        total_working_hours[subject] += int(self._problem.slot_duration(slt_idx) / SCHOOL_HOUR)
                        table |= {slot: [subject, teacher]}

        for subject, hours in total_working_hours.items():
            if self._problem.curriculum_contains(level, subject):
                print(f"Total teaching hours for {subject}: {hours}")
        df = pd.DataFrame.from_dict(
            table, orient="index", columns=["subject", "teacher"]
        )
        print(tabulate(df, headers="keys", tablefmt="psql"))

    def print_school_schedule(self):
        table = {}
        print("\nSchool:")
        for slt_idx, slot in enumerate(self._problem.time_slots):
            tmp = ""
            for lvl_idx, level in enumerate(self._problem.levels):
                for sub_idx, subject in enumerate(self._problem.subjects):
                    for tch_idx, teacher in enumerate(self._problem.teachers):
                        key = (lvl_idx, sub_idx, tch_idx, slt_idx)
                        if self._solver.BooleanValue(self._assignment[key]):
                            tmp += f" {level}:({subject},{teacher})"
            table |= {slot: [tmp]}
        df = pd.DataFrame.from_dict(
            table, orient="index", columns=["class:(subject, teacher)"]
        )
        print(tabulate(df, headers="keys", tablefmt="psql"))

    def solve(self):
        print("Solving")

        # --------------------------------------------------------------------
        # Create Solver and Printer
        # --------------------------------------------------------------------
        self._solver = cp_model.CpSolver()

        solution_printer = SchoolSchedulingSatSolutionPrinter()

        # --------------------------------------------------------------------
        # Solve the problem
        # --------------------------------------------------------------------
        status = self._solver.Solve(self._model, solution_printer)
        print("\nResults:")
        print("Status: ", self._solver.StatusName(status))

        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            print("\n")
            print("-" * 75)
            print("Teachers")
            print("-" * 75)
            for teacher_idx in self._all_teachers:
                self.print_teacher_schedule(teacher_idx)

            print("\n")
            print("-" * 75)
            print("Classes")
            print("-" * 75)
            for level_idx in self._all_levels:
                self.print_class_schedule(level_idx)

            self.print_school_schedule()

        print("\nStatistics:")
        print("Branches: %f" % self._solver.NumBranches())
        print("Conflicts: %f" % self._solver.NumConflicts())
        print("Problem solved in %f seconds" % self._solver.WallTime())

# 4. Input Data

In [415]:
# Classes
LEVELS = [
    "1-A",
    "1-B",
    "2-A",
    "2-B",
    "3-A",
]

# Subjects
SUBJECTS = [
    "English",
    "Math",
    "History",
    "Science",
    "Sport",
]

# The required number of SCHOOL_HOURS per class and subject.
CURRICULUM = {
    ("1-A", "English"): 3,
    ("1-A", "Math"): 3,
    ("1-A", "History"): 2,
    ("1-A", "Sport"): 2,
    ("1-B", "English"): 3,
    ("1-B", "Math"): 3,
    ("1-B", "History"): 2,
    ("1-B", "Sport"): 2,
    ("2-A", "English"): 4,
    ("2-A", "Math"): 2,
    ("2-A", "History"): 2,
    ("2-B", "English"): 4,
    ("2-B", "Math"): 2,
    ("2-B", "History"): 2,
    ("3-A", "English"): 2,
    ("3-A", "Math"): 4,
    ("3-A", "Science"): 2,
}

# Specific teachers can be assigned directly to a class/subject combination.
PREFERRED_TEACHERS = {
    ("1-A", "English"): ["T2"],
    ("1-B", "English"): ["T4"],
}

# An explicit assignment of class/subject/time-slot to a teacher.
PREFERRED_TEACHERS_TIME_SLOTS = {
    ("1-A", "Sport", "Monday:07:10-07:55"): ["T6"],
    ("1-A", "Sport", "Monday:08:00-08:45"): ["T6"],
    ("1-B", "Sport", "Tuesday:07:10-07:55"): ["T7"],
}

# Teacher maximum SCHOOL_HOURS for a scheduling period.
TEACHERS = {
    "T1": 14,
    "T2": 12,
    "T3": 12,
    "T4": 14,
    "T5": 2,
    "T6": 2,
    "T7": 2,
}

# Available locations and rooms and an associated cost.
LOCATION_ROOM = {
    ("A", "1"): 0,
    ("A", "2"): 0,
    ("A", "3"): 0,
    ("A", "4"): 0,
    ("A", "5"): 0,
    ("A", "6"): 0,
    ("A", "7"): 0,
    ("A", "8"): 0,
    ("A", "9"): 0,
    ("A", "10"): 0,
}

# Subject list of teachers who can teach it
SPECIALTIES = {
    "English": ["T2", "T4"],
    "Math": ["T1", "T4"],
    "History": ["T3", "T4"],
    "Science": ["T5"],
    "Sport": ["T6", "T7"],
}

# The Schedule. It defines the available time-slots [min] for a complete planning period.
TIME_SLOTS = {
    "Monday:07:10-07:55": SCHOOL_HOUR,
    "Monday:08:00-08:45": SCHOOL_HOUR,
    "Monday:09:00-09:45": SCHOOL_HOUR,
    "Monday:10:30-11:15": SCHOOL_HOUR,
    "Monday:11:25-12:10": SCHOOL_HOUR,
    "Tuesday:07:10-07:55": SCHOOL_HOUR,
    "Tuesday:08:00-08:45": SCHOOL_HOUR,
    "Tuesday:09:00-09:45": SCHOOL_HOUR,
    "Tuesday:10:30-11:15": SCHOOL_HOUR,
    "Tuesday:11:25-12:10": SCHOOL_HOUR,
    "Wednesday:07:10-07:55": SCHOOL_HOUR,
    "Wednesday:08:00-08:45": SCHOOL_HOUR,
    "Wednesday:09:00-09:45": SCHOOL_HOUR,
    "Wednesday:10:30-11:15": SCHOOL_HOUR,
    "Wednesday:11:25-12:10": SCHOOL_HOUR,
    "Thursday:07:10-07:55": SCHOOL_HOUR,
    "Thursday:08:00-08:45": SCHOOL_HOUR,
    "Thursday:09:00-09:45": SCHOOL_HOUR,
    "Thursday:10:30-11:15": SCHOOL_HOUR,
    "Thursday:11:25-12:10": SCHOOL_HOUR,
    "Friday:07:10-07:55": SCHOOL_HOUR,
    "Friday:08:00-08:45": SCHOOL_HOUR,
    "Friday:09:00-09:45": SCHOOL_HOUR,
    "Friday:10:30-11:15": SCHOOL_HOUR,
    "Friday:11:25-12:10": SCHOOL_HOUR,
}

# 5. Solving the Problem

In [416]:
problem = SchoolSchedulingProblem(
    LEVELS,
    SUBJECTS,
    CURRICULUM,
    TEACHERS,
    PREFERRED_TEACHERS,
    PREFERRED_TEACHERS_TIME_SLOTS,
    SPECIALTIES,
    TIME_SLOTS,
)
solver = SchoolSchedulingSatSolver(problem)

solver.solve()

There are 4375 variable assignments.
Solving
Solution #0, objective: 0.0

Results:
Status:  OPTIMAL


---------------------------------------------------------------------------
Teachers
---------------------------------------------------------------------------

Teacher: T1
Total teaching hours: 14
Unallocated teaching hours: 0
+-----------------------+---------+-----------+
|                       | class   | subject   |
|-----------------------+---------+-----------|
| Monday:07:10-07:55    | 2-A     | Math      |
| Monday:08:00-08:45    | 2-A     | Math      |
| Monday:09:00-09:45    | 3-A     | Math      |
| Monday:10:30-11:15    | 3-A     | Math      |
| Monday:11:25-12:10    | 3-A     | Math      |
| Tuesday:07:10-07:55   | 3-A     | Math      |
| Tuesday:08:00-08:45   | 2-B     | Math      |
| Tuesday:09:00-09:45   | 2-B     | Math      |
| Wednesday:09:00-09:45 | 1-B     | Math      |
| Wednesday:10:30-11:15 | 1-B     | Math      |
| Thursday:07:10-07:55  | 1-B     | Math     